In [4]:
#pip install transformers faiss-cpu torch sentence-transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from accelerate import infer_auto_device_map, dispatch_model

# Path to the model directory
model_path = "/Users/sheebamoghal/Downloads/rag/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model with disk offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # Use float16 to save memory
    low_cpu_mem_usage=True      # Optimize memory during loading
)

# Generate a device map with disk offloading
device_map = infer_auto_device_map(
    model,
    max_memory={"cpu": "48GiB", "disk": "128GiB"}  # Adjust memory limits based on your system
)

# Dispatch the model with disk offloading
model = dispatch_model(
    model,
    device_map=device_map,
    offload_dir="/tmp"  # Specify a directory for disk offloading
)

# Initialize the pipeline
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map=device_map
)

# Generate text
input_text = "Hey, how are you doing today?"
result = text_generator(input_text, max_length=50, num_return_sequences=1)

# Print the generated text
print(result[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]